In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from datetime import datetime, timedelta
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import os
print(os.getcwd())

In [ ]:
path = 'L1_Train.csv'
df = pd.read_csv(path)

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
print(df.head())
print(df.info())

In [ ]:
testTime = df['DateTime'][0]
print(testTime.time())

In [ ]:
startTime = datetime.strptime('09:00:00', '%H:%M:%S').time()
endTime = datetime.strptime('17:00:00', '%H:%M:%S').time()
df_filt_9_to_17 = df[(df['DateTime'].dt.time >= startTime) & (df['DateTime'].dt.time <= endTime)]
# [element for element in df if  startTime < element[1].time() < endTime]
print(df_filt_9_to_17.info())

In [ ]:
# df_filt_9_to_17.to_csv('Checking_File.csv',index=False, header = True)  no problem in df_filt_9_to_17

In [ ]:
interval = pd.Timedelta(minutes=10)
def generate_expected_times(start_time, end_time, interval):
    times = []
    current_time = pd.Timestamp.combine(pd.Timestamp.today(), start_time)
    end_timestamp = pd.Timestamp.combine(pd.Timestamp.today(), end_time)
    while current_time <= end_timestamp:
        times.append(current_time.time())
        # print(current_time.time(), interval)
        current_time += interval
    return times

expected_times = generate_expected_times(startTime, endTime, interval)

In [ ]:
print(expected_times[1]) #from 9 to 17 every 10
check_list = np.zeros(49)
print(check_list.shape, len(expected_times))
# print(check_list, expected_times)

In [ ]:
nowday = datetime.strptime('2024-01-01', '%Y-%M-%d').date()
print(nowday)

In [ ]:
nowday = datetime.strptime('2024-08-01', '%Y-%M-%d').date()
accepted_Days = []
check_list = np.zeros(49)
c = 0
for idx, obj in enumerate(df['DateTime']):
    if obj.date() != nowday or idx == len(df['DateTime'])-1:
        v = sum(check_list)
        # print(v)
        if v == 47:
            accepted_Days.append(nowday)    #date() for YMd, date for all
        nowday = obj.date()
        check_list = np.zeros(49)
        c = 0
    else:
        if expected_times[c] < obj.time() < expected_times[c+1] and c < 47:
            check_list[c] += 1
            if c < 47:
                c += 1
print(len(accepted_Days))
print(accepted_Days)  # Jan 1 2 3 4 5 7 8 9 10 12 13 16 18 20 21 22 23 24 25 27 28 29 30 31

In [ ]:
df['DateTime'].dt.date.head()

In [ ]:
for a, i in enumerate(accepted_Days):
    print(i)
    if a > 10:
        break

In [ ]:
# df_new.to_csv('Checking_File.csv',index=False, header = True)  #no problem in df_filt_9_to_17

In [ ]:
accepted_days_set = set(accepted_Days)
df_fill_10min = df_filt_9_to_17[df_filt_9_to_17['DateTime'].dt.date.isin(accepted_days_set)]

# 查看篩選後的結果
print(len(accepted_days_set))
print(df_fill_10min.info())

In [ ]:
print(df_filt_9_to_17.info())
print(df_fill_10min.info())

In [ ]:
# 設定 DateTime 為索引
df_new = df_fill_10min.copy()
print(df_new.info())
# df_new.to_csv('Checking_File.csv', index=False, header = True)
df_new.set_index('DateTime', inplace=True)
# df_new.to_csv('Checking_File2.csv', index=False, header = True)
# 每 10 分鐘重採樣並計算平均
df_resampled = df_new.resample('10min').mean().dropna()
# df_new.to_csv('Checking_File3.csv', index=False, header = True)
# df_resampled.to_csv('Checking_File4.csv', index=False, header = True)
# 如果需要重設索引
df_resampled.reset_index(inplace=True)
# df_resampled.to_csv('Checking_File5.csv', index=False, header = True)
print(df_resampled.info())
#5136 / 48 = 107

In [ ]:
df_resampled.loc[55]

In [ ]:
df_resampled.columns

In [ ]:
df_resampled['timeSeries'] = df_resampled['DateTime'].dt.hour + df_resampled['DateTime'].dt.minute/60

In [ ]:
df_resampled.head()

In [ ]:
df_resampled = df_resampled[['DateTime', 'timeSeries', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']]

In [ ]:
df_resampled.head()

In [ ]:
df_resampled = df_resampled.drop(columns='DateTime')
df_resampled.head()

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, out_size = 1, num_layers = 1, dropout = 0.1):
        super().__init__()
        self.lstm_1_layer = nn.LSTM(input_size, hidden_size1, num_layers)
        # self.connect1 = nn.Linear(hidden_size1, hidden_size2)
        self.lstm_2_layer = nn.LSTM(hidden_size1, hidden_size2, num_layers)
        self.output_layer = nn.Linear(hidden_size2, out_size)
        self.dropout = nn.Dropout(dropout)
        self.hidden = (torch.zeros(1,1,hidden_size1), torch.zeros(1,1,hidden_size1))
    def forward(self, seq):
        seq, (hidden_state1, cell_state1) = self.lstm_1_layer(seq)
        seq = self.dropout(seq)
        # seq = self.connect1(seq)
        # seq = self.dropout(seq)
        seq, (hidden_state2, cell_state2) = self.lstm_2_layer(seq)
        seq = self.dropout(seq)
        output = self.output_layer(seq)
        return output

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data  # 假設是形狀為 (num_samples, n_steps, n_features) 的張量
        self.targets = targets  # 假設是形狀為 (num_samples, target_size) 的張量

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx].unsqueeze(-1)
        return sample, target

In [ ]:
import torch

# 檢查是否有可用的 GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # 如果有 GPU，則使用它
    print("GPU is available.")
else:
    device = torch.device("cpu")  # 否則使用 CPU
    print("GPU is not available, using CPU.")

In [ ]:
df_values = df_resampled.values
##add normalization
LSTM_MinMaxModel = MinMaxScaler().fit(df_values)
data_tensor = LSTM_MinMaxModel.transform(df_values)

In [ ]:
data_tensor = df_resampled.values
X_data_tensor=data_tensor[:, :7]
Y_data_tensor=data_tensor[:, 7]
Y_data_tensor = Y_data_tensor.reshape(-1, 1)
print(X_data_tensor.shape, Y_data_tensor.shape)

NX = True
NY = True
##normalization
if NX:
    scaler_X = MinMaxScaler().fit(X_data_tensor)
    train_data_n = scaler_X.transform(X_data_tensor)
else:
    train_data_n = X_data_tensor
if NY:
    scaler_Y = MinMaxScaler().fit(Y_data_tensor)
    train_targets_n = scaler_Y.transform(Y_data_tensor)
else:
    train_targets_n = Y_data_tensor

print(train_data_n.shape, train_targets_n.shape)

In [ ]:
## expand numpy to sequence
def to_sequence(data, stamp):
    data_seq = []
    for i in range(len(data) - stamp):
        seq = data[i:i + stamp]
        data_seq.append(seq)
    return np.array(data_seq)
    
stamp = 48
train_data = to_sequence(train_data_n, stamp)
train_targets = to_sequence(train_targets_n, stamp).squeeze()
print(train_data.shape, train_targets.shape)

In [ ]:
# train_data = train_data_n.reshape(107,48,7)  # 形狀應為 (num_samples, n_steps, n_features)
# train_targets = train_targets_n.reshape(107,48)   # 形狀應為 (num_samples, target_size, 1)

train_data_tensor = torch.tensor(train_data, dtype=torch.float32) #.to(device)
train_targets_tensor = torch.tensor(train_targets, dtype=torch.float32) #.to(device)
print(train_data_tensor.shape, train_targets_tensor.shape)

# 創建數據集和數據加載器
dataset = TimeSeriesDataset(train_data_tensor, train_targets_tensor)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)  # 設定 batch_size 和 shuffle

In [ ]:
for train_input, train_target in train_loader:
    print(train_input.shape, train_target.shape)
    break
print(train_input[0][0])
print(train_target[0][0])

In [ ]:
input_size = 7  # 單變量的時間序列
hidden_size1 = 128
hidden_size2 = 64
out_size = 1
num_layers = 1
dropout = 0.2
learning_rate = 0.001
n_epochs = 100
losses = []
print(len(losses))
output_list = []
# 模型初始化
model = LSTM(input_size, hidden_size1, hidden_size2, out_size, num_layers, dropout).to(device)
for train_input, train_target in train_loader:
    print(train_input.shape, train_target.shape)
    break
criterion = nn.MSELoss()  # 使用均方誤差作為損失函數
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(n_epochs):
    model.train()  # 設定模型為訓練模式
    for train_input, train_target in train_loader:  # 從 DataLoader 獲取每個批次
        train_input, train_target = train_input.to(device), train_target.to(device)
        optimizer.zero_grad()  # 清空之前的梯度

        output = model(train_input)  # 將掩碼傳遞給模型的前向方法train_target,
        loss = criterion(output, train_target)  # 計算損失
        output_list.append(output)
        # print(output.shape, train_target.shape)
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新參數

    losses.append(loss)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}/{n_epochs}, Loss: {loss.item():.4f}')

# 打印最後的損失
print(f'Final Loss: {loss.item():.4f}')
print(len(losses))
torch.cuda.empty_cache()
##(unnormalize)
##dropout=0.2  -> loss:6
##dropout=0.1  -> loss:5.7
##dropout=0.1  -> loss:5.4

In [ ]:
# 打印最後的損失
print(len(losses))
x_axis = np.arange(1, n_epochs+1)
losses_tensor  = torch.stack(losses)
loss_np = losses_tensor.cpu().detach().numpy()
plt.plot(x_axis, loss_np)
plt.show()

In [ ]:
days = 5
x_test = data_tensor[:stamp, :7]
y_test = data_tensor[:stamp, 7]
y_test = y_test.reshape(-1,1)

print(x_test.shape, y_test.shape)
if NX:
    test_data_n = scaler_X.transform(x_test)
else:
    test_data_n = x_test

test_data_n = np.expand_dims(x_test, axis=0)

test_data_n = torch.from_numpy(test_data_n).float().to(device)
print(test_data_n.shape)

out = []
model.eval()
with torch.no_grad():
    out = model(test_data_n)
    out_np  = out.cpu().numpy().squeeze().reshape(-1, 1)
    # y_test = y_test.squeeze().reshape(-1, 1)
    print(out_np.shape, y_test.shape)
if NY:
    # out_np = scaler_Y.inverse_transform(out_np)
    # y_test = scaler_Y.inverse_transform(y_test)
    out_np.squeeze()
    
    result = abs(y_test-out_np)
else:
    out_np.squeeze()
    result = abs(y_test-out_np)
print(out_np.shape, y_test.shape)

In [ ]:
result_re = result.reshape(result.shape[0]*result.shape[1])

In [ ]:
y_test = y_test.reshape(-1,1)
plt.plot(y_test)

In [ ]:
show_a = y_test.squeeze()
show_b = out_np.squeeze()
plt.plot(result_re, c = "r")
plt.plot(show_a, c = "b")
plt.plot(show_b, c = "g")
plt.show()
print(sum(result_re))

In [ ]:
#LSTM 2 layers
##(unnormalize)
##dropout=0.2  -> loss:6
##dropout=0.1  -> loss:5.7
##dropout=0  -> loss:5.4
# abs(583.7734)
##(normalize x,y)  epoch 100
##dropout=0.2 -> Loss: 0.0228
##dropout=0.1 -> Loss: 0.0236
##dropout=0 -> Loss: 0.0226
# abs(5826.7388)
##(normalize x only) epoch 300
##dropout=0.2 -> Loss: 6.6351
##dropout=0.1 -> Loss: 6.2561
##dropout=0 -> Loss: 6.1234
# abs(867)


##transformer d_m=256, nhead=4, encoderlayer=3, dim_feed=1024, dropout= 0.1
##(unnormalize) epoch 200
#dropout= 0.1 -> Loss: 19.1384
# abs(794.7)
##(normalize x only)  epoch 200
#dropout= 0.1 -> Loss: 21.35
# abs(858)
##transformer d_m=512, nhead=8, encoderlayer=6, dim_feed=2048, dropout= 0.1
##(unnormalize) epoch 60
#dropout= 0.1 -> Loss: 6.9705
# abs(483.7)


# lstm hidden_size1 = 128, hidden_size2 = 64 ,out_size = 1, num_layers = 1, 
# dropout = 0.2, learning_rate = 0.001, n_epochs = 100
# NY=False on testing sum lastDAy, 450

15028.978

# transformer NY+NX=True, input_dim=7, d_model = 256, nhead = 4, num_encoder_layers = 3, dim_feedforward = 1024 , 
# dropout = 0.1, learning_rate = 0.00001, n_epochs = 150
#sum lastDAy, 1865.8014

In [ ]:
torch.cuda.empty_cache()

In [ ]:
####transformer part:


input_dim = 7  # 單變量的時間序列
d_model = 256  #  Transformer embedding的維度
nhead = 4     # 多頭注意力的頭數
num_encoder_layers = 3  # Encoder的層數
dim_feedforward = 1024  #Feedforward隱藏層的維度
dropout = 0.1
learning_rate = 0.00001
n_epochs = 150

# 模型初始化
model = TimeSeriesTransformer(input_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout).to(device)
for train_input, train_target in train_loader:
    print(train_input.shape, train_target.shape)
    break
criterion = nn.MSELoss()  # 使用均方誤差作為損失函數
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

trans_loss = []
for epoch in range(n_epochs):
    model.train()  # 設定模型為訓練模式
    for train_input, train_target in train_loader:  # 從 DataLoader 獲取每個批次
        train_input, train_target = train_input.to(device), train_target.to(device)
        optimizer.zero_grad()  # 清空之前的梯度

        output = model(train_input,train_target)  # 將掩碼傳遞給模型的前向方法train_target,
        loss = criterion(output, train_target)  # 計算損失
        # print(output.shape, train_target.shape)
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新參數
    trans_loss.append(loss)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}/{n_epochs}, Loss: {loss.item():.4f}')

# 打印最後的損失
print(f'Final Loss: {loss.item():.4f}')
torch.cuda.empty_cache()

In [ ]:
print(len(trans_loss))
x_axis = np.arange(1, n_epochs+1)
losses_tensor  = torch.stack(trans_loss)
loss_np = losses_tensor.cpu().detach().numpy()
plt.plot(loss_np)
plt.show()

In [ ]:
y_test

In [ ]:
days = 5
x_test = data_tensor[:stamp, :7]
y_test = data_tensor[:stamp, 7]
y_test = y_test.reshape(-1,1)

print(x_test.shape, y_test.shape)
if NX:
    x_test = scaler_X.transform(x_test)
else:
    x_test = x_test
if NY:
    y_test = scaler_Y.transform(y_test)
else:
    y_test = Y_data_tensor
x_test = np.expand_dims(x_test, axis=0)
y_test = np.expand_dims(y_test, axis=0)  #or axis = 1?

x_test = torch.from_numpy(x_test).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)
print(x_test.shape, y_test.shape)

out = []
model.eval()
with torch.no_grad():
    out = model(x_test, y_test)
    out_np  = out.cpu().numpy().squeeze().reshape(1, -1)
    y_test = y_test.cpu().numpy().squeeze().reshape(1, -1)
print(out_np.shape, y_test.shape)
if NY:
    ren_out_np = scaler_Y.inverse_transform(out_np)
    y_test = scaler_Y.inverse_transform(y_test)
    ren_out_np.squeeze()
    result = abs(y_test-ren_out_np)
else:
    ren_out_np = out_np
    result = abs(y_test-out_np)

    

In [ ]:
print(train_data_tensor.shape)
print(x_test.shape)
print(train_targets_tensor.shape)
print(y_test.shape)
print(ren_out_np.shape)

In [ ]:
result_re = result.reshape(result.shape[0]*result.shape[1])

In [ ]:
show_a = y_test.reshape(1,-1).squeeze()
show_b = ren_out_np.reshape(1,-1).squeeze()
# plt.plot(result_re, c = "r")
plt.plot(show_a, c = "b")
plt.plot(show_b, c = "g")
plt.show()
print(sum(result_re))

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        # 創建一個長度為 max_len 的位置編碼矩陣
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶數位置
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇數位置
        pe = pe.unsqueeze(0).transpose(0, 1)  # 調整維度
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x


class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model)
        self.input_layer = nn.Linear(input_dim, d_model)  # 將輸入轉換為 d_model 維度
        self.target_input_layer = nn.Linear(1, d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, 
                                          dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(d_model, 1)  # 回歸輸出

    def forward(self, src, tgt):
        
        src = self.input_layer(src)  # 映射到 d_model 維度
        src = self.positional_encoding(src)  # 添加位置編碼

        tgt = self.target_input_layer(tgt)  # 或者使用一個不同的線性層，例如 self.tgt_layer = nn.Linear(target_input_dim, d_model)
        tgt = self.positional_encoding(tgt)  # 添加位置編碼
    
        output = self.transformer(src, tgt)   #(src, tgt, src_key_padding_mask=mask)
        output = self.fc_out(output)  # 預測最後一個時間步的值
        return output
